# Count Min Sketch Implementation

Description: <br>
Count Min Sketch (12.2.2.2, Pg 403- 405 Aggarwal)

## 1. Step by step definition

In [1]:
# importing libraries
# https://pypi.org/project/mmh3/
import mmh3 # can be used to hash strings
import random
import math

In [2]:
# parameters
# w
length = 7
# d
noHashFunctions = 5

In [3]:
#initialize the |noHashFunctions| arrays with 0
arrays = []

for i in range(0,noHashFunctions):
    array = [0] * length
    arrays.append(array)
    
print(arrays)

[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]


In [4]:
#we need to define pairwise independent hash functions that map an item from 0 to length - 1
def hashItem(item,seed):
    #returns a 32-bit unsigned int 
    #seed is used to create independence
    hashCode = mmh3.hash(str(item),seed, signed = False)
    #maxValue of 32-bit unsigned int 
    maxValue = 4294967295
    #normalize it between 0 and length-1
    normalize = hashCode/maxValue * (length-1)
    #round ensures even splits
    #round(3.6) = 4 while int(3.6) = 3 
    return round(normalize)

In [5]:
#counting an item
def count(item):
    for i in range(0,noHashFunctions):
        index = hashItem(item = item, seed = i)
        arrays[i][index] += 1

In [6]:
#counting an item
item_1 = 2000
item_2 = "countmin"
count(item_1)
count(item_2)
count(item_2)
print(arrays)

[[0, 0, 0, 1, 2, 0, 0], [0, 0, 2, 1, 0, 0, 0], [0, 0, 2, 0, 1, 0, 0], [0, 3, 0, 0, 0, 0, 0], [0, 1, 0, 0, 2, 0, 0]]


As shown, array 4 has a count collision.

In [7]:
#finding the count of an item
def getCount(item):
    #taking the min reduces the effect of collisions
    minCount = float('inf')

    for i in range(0,noHashFunctions):
        index = hashItem(item = item, seed = i)
        count = arrays[i][index]

        if count < minCount:
            minCount = count
    
    return minCount

In [8]:
print(getCount(item_1))
print(getCount(item_2))

1
2


## 2. Class Definition

Let $||count||_1$ be the sum of all counts stored in the data structure, i.e. the sum of values in one row of the sketch. The central guarantee CMS provides is then the following:<br>
Theorem: With a probability of $1−\delta$, the error is at most $\epsilon∗||count||_1$. Concrete values for these error bounds $\epsilon$ and $\delta$ can be freely chosen by setting:<br>
$w=⌈\frac{e}{\epsilon}⌉$ and $d=⌈ln\frac{1}{\delta}⌉$

### mmh3 version

In [41]:
import mmh3
import math

class CountMinSketch():
    def __init__(self,delta = 0.001,epsilon= 0.005):
        self.length = math.ceil(math.e/epsilon)
        self.noHashFunctions = math.ceil(math.log((1/delta), math.e))
        self.arrays = []
        # stores already seen hashes to avoid recomputation
        self.cache = {}
        self.counter = 0

        for i in range(0,self.noHashFunctions):
            array = [0] * self.length
            self.arrays.append(array)
            
    def hashItem(self,item,seed):
        key = str(item) + " " + str(seed)
        if key in self.cache:
            return self.cache[key]
        
        print(key)
        
        self.counter += 1
        hashCode = mmh3.hash(str(item),seed, signed = False)
        maxValue = 4294967295
        normalize = hashCode/maxValue * (self.length-1)
        self.cache[key] = round(normalize)
        return round(normalize)
    
    def count(self, item):
        for i in range(0,self.noHashFunctions):
            index = self.hashItem(item = item, seed = i)
            self.arrays[i][index] += 1
            
    def getCount(self,item):
        minCount = float('inf')

        for i in range(0,self.noHashFunctions):
            index = self.hashItem(item = item, seed = i)
            count = self.arrays[i][index]

            if count < minCount:
                minCount = count

        return minCount

In [42]:
cms = CountMinSketch()
print(cms.length,cms.noHashFunctions)

544 7


In [37]:
#counting an item
item_1 = 2000
item_2 = "countmin"

In [43]:
cms.count(item_1)

for i in range(0,20):
    cms.count(item_2)

In [44]:
print(cms.getCount(item_1))
print(cms.getCount(item_2))

1
20


### Rolling polynomial hashing version

Assumptions:<br>
- All strings lowercase<br>
- Numbers(0-9) and Letters(a-z) only 

In [14]:
import string
import random

# this will enable us to get different independent mappings for different seeds
def randomMap(seed):
    mapping = {}
    characters =  string.ascii_lowercase + '0123456789'
    characters = list(characters)
    random.Random(seed).shuffle(characters)

    for i,char in enumerate(characters):
        mapping[char] = i + 1

    return mapping

# will return integer value to be used in the polynomial rolling algorithm
def generateAsciiMapping(item,seed):
    mapping = randomMap(seed)

    if not item.isalnum():
        raise Exception("Not alphanumeric")
        
    l = list(item)
    n = []
    
    for char in l:
        char = char.lower()
        n.append(mapping[char])
    
    return n

We have 36 characters. Therefore we need a prime > 36. To be able to map these characters to the arrays in our count min sketch, we also need a mod which is equal to the size of each array we use 

hash(s) = $(\sum_{i= 0} ^ {n-1} s[i].p^i)$mod m

In [15]:
# polynomial rolling hash algorithm
def generatePolyhash(string,mod,seed):
    p_power = 1
    prime = 39
    string = str(string)
    mapToInteger = generateAsciiMapping(string,seed)
    value = 0
    
    for no in mapToInteger:
        value = (value + (no * p_power)) % mod
        p_power = (p_power * prime) % mod 
    
    return value

All this was exported to a file called polyhash.py

In [16]:
import polyhash 
import math

class CountMinSketchPoly():
    def __init__(self,delta = 0.001,epsilon= 0.005):
        self.length = math.ceil(math.e/epsilon)
        self.noHashFunctions = math.ceil(math.log((1/delta), math.e))
        self.arrays = []
        # stores already seen hashes to avoid recomputation
        self.cache = {}

        for i in range(0,self.noHashFunctions):
            array = [0] * self.length
            self.arrays.append(array)
            
    def hashItem(self,item,seed):
        key = str(item) + " " + str(seed)
        if key in self.cache:
            return self.cache[key]
        
        hashCode = polyhash.generatePolyhash(str(item),self.length,seed)
        self.cache[key] = hashCode
        return hashCode
    
    def count(self, item):
        for i in range(0,self.noHashFunctions):
            index = self.hashItem(item = item, seed = i)
            self.arrays[i][index] += 1
            
            
    def getCount(self,item):
        minCount = float('inf')

        for i in range(0,self.noHashFunctions):
            index = self.hashItem(item = item,seed = i)
            count = self.arrays[i][index]

            if count < minCount:
                minCount = count

        return minCount

In [17]:
cmsPoly = CountMinSketchPoly()
print(cmsPoly.length,cmsPoly.noHashFunctions)

544 7


In [18]:
#counting an item
item_1 = 2000
item_2 = "countmin"

In [19]:
cmsPoly.count(item_1)

for i in range(0,20):
    cmsPoly.count(item_2)

In [20]:
print(cmsPoly.getCount(item_1))
print(cmsPoly.getCount(item_2))

1
20


# 4 . Analysis on real data

We will compare our approximation algorithms with the exact frequency counting algorithm on the webdocs dataset, which contains a high number of distinct items and may use a lot of space

In [21]:
# EXEMPLARY SOLUTION HOMEWORK 1
import sys
import os
from collections import defaultdict

def exact_algorithm(file_path):
    """Run exact frequent items algorithm"""
    M = defaultdict(int)
    dataset_size = 0
    output = list()

    with open(file_path, "r") as dataset:
        # Process the transactions as we get them
        for transaction in dataset:
            dataset_size += 1
            # Get the transaction as list of ints
            transaction = [
                int(item) for item in transaction.split() if item.isnumeric()
            ]
            # Using defaultdict properties of initializing the value for a
            # non-existing key to 0
            for item in transaction:
                M[item] += 1

    for key in M:
            output.append((key, M[key]))

    return output

In [46]:
import sys
import os
from collections import defaultdict

def cms_algorithm(file_path,hashFunction = "mmh3"):
    dataset_size = 0
    output = list()
    M = set()
    cms = CountMinSketch()
    
    if hashFunction == "poly":
        cms = CountMinSketchPoly()

    print("Using only" ,cms.length, "by", cms.noHashFunctions, "entries")
    
    with open(file_path, "r") as dataset:
        # Process the transactions as we get them
        for transaction in dataset:
            dataset_size += 1
            # Get the transaction as list of ints
            transaction = [
                int(item) for item in transaction.split() if item.isnumeric()
            ]

            for item in transaction:
                cms.count(item)
                M.add(item)

    for key in M:
        output.append((key, cms.getCount(key)))
    
    print(cms.counter)

    return output

In [32]:
from time import perf_counter 
file = "./accidents.dat"

In [30]:
start = perf_counter()  
output_exact = exact_algorithm(file)
end = perf_counter()
executionTimeMsExact = (end - start) * 1000.0

In [47]:
start = perf_counter()  
output_cms_mmh3 = cms_algorithm(file,"mmh3")
end = perf_counter()
executionTimeMsMmh3 = (end - start) * 1000.0

Using only 544 by 7 entries
3276


In [48]:
len(output_cms_mmh3)

468

In [49]:
3276/468

7.0

In [ ]:
start = perf_counter()  
output_cms_poly = cms_algorithm(file,"poly")
end = perf_counter()
executionTimeMsPoly = (end - start) * 1000.0

In [ ]:
len(output_exact)
executionTimeMsExact

## Observations

In [50]:
def avg_difference(output_1,output_2):
    difference = 0
    index = 0
    length = len(output_1)
    
    for i in range(0, length):
        if output_1[i][0] != output_2[i][0]:
            raise Exception("Keys do not match")
        else:
            difference += abs(output_1[i][1] - output_2[i][1])
            
    # average difference
    return difference/length

In [52]:
avg_difference(output_exact,output_cms_mmh3)

0.14316239316239315

In [178]:
avg_difference(output_exact,output_cms_poly)

0.9743589743589743

With a delta = 0.001 and epsilon= 0.005, the count min sketch only used 544 by 7 arrays. The exact algorithm 